In [ ]:
import os
import openai
import dotenv

dotenv.load_dotenv()

endpoint = os.environ["OLLAMA_API_URL"]
api_key = os.environ["OLLAMA_API_KEY"]
deployment = "functiongemma"

In [ ]:
if endpoint == "" or api_key == "" or deployment == "":
    print("Bitte trage die API Daten in .env Datei ein. ")

In [ ]:
client = openai.OpenAI(
        base_url=endpoint,
        api_key=api_key,
)

## Senden von Nachrichten

In [ ]:
message_text = [
    {"role":"system","content":"You are a helpful assistant that answers questions about Telekom MMS."},
    {"role":"user","content":"What is Telekom MMS?"},]

completion = client.chat.completions.create(
  model=deployment,
  messages = message_text,
  max_tokens=60
)

print(completion.choices[0].message.content)

## Streaming von Antworten

In [ ]:
completion = client.chat.completions.create(
  model=deployment,
  messages = message_text,
  max_tokens=300, stream=True
)

for element in completion: 
    if len(element.choices) > 0:    
        print(element.choices[0].delta.content, end="", flush=True)

## Structured Outputs - ein JSON als Antwort

In [ ]:
from pydantic import BaseModel

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.beta.chat.completions.parse(
    model=deployment,
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Tina and John meet on December 12, 2024 for a tombola."},
    ],
    response_format=CalendarEvent,
    temperature=0.0,
)

event = completion.choices[0].message.parsed

print(event)